<a href="https://colab.research.google.com/github/GTmycology/dreambooth-colab/blob/main/fast-DreamBooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab Pro notebook from https://github.com/TheLastBen/fast-stable-diffusion. [ComfyUI Colab](https://colab.research.google.com/github/TheLastBen/fast-stable-diffusion/blob/main/fast_stable_diffusion_ComfyUI.ipynb)**

In [42]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [45]:
import os

BASE_DIR = "/content/gdrive/MyDrive/Loras/playful_origins_v4"
INSTANCE_DIR = f"{BASE_DIR}/dataset"
CAPTIONS_DIR = f"{BASE_DIR}/captions"
OUTPUT_DIR = f"{BASE_DIR}/output"


MODEL_NAME = "runwayml/stable-diffusion-v1-5"
PT = "a photo of [your-unique-identifier] person"

# Create folders if they don't exist
for d in [INSTANCE_DIR, CAPTIONS_DIR, OUTPUT_DIR]:
    os.makedirs(d, exist_ok=True)

print("✅ Path check complete:")
print("Instance dir:", INSTANCE_DIR, "->", os.path.exists(INSTANCE_DIR))
print("Captions dir:", CAPTIONS_DIR, "->", os.path.exists(CAPTIONS_DIR))
print("Output dir:", OUTPUT_DIR, "->", os.path.exists(OUTPUT_DIR))


✅ Path check complete:
Instance dir: /content/gdrive/MyDrive/Loras/playful_origins_v4/dataset -> True
Captions dir: /content/gdrive/MyDrive/Loras/playful_origins_v4/captions -> True
Output dir: /content/gdrive/MyDrive/Loras/playful_origins_v4/output -> True


In [46]:
# Create folders if they don’t exist
for d in [INSTANCE_DIR, CAPTIONS_DIR, OUTPUT_DIR]:
    os.makedirs(d, exist_ok=True)

print("✅ Path check complete:")
print("Instance dir:", INSTANCE_DIR, "->", os.path.exists(INSTANCE_DIR))
print("Captions dir:", CAPTIONS_DIR, "->", os.path.exists(CAPTIONS_DIR))
print("Output dir:", OUTPUT_DIR, "->", os.path.exists(OUTPUT_DIR))


✅ Path check complete:
Instance dir: /content/gdrive/MyDrive/Loras/playful_origins_v4/dataset -> True
Captions dir: /content/gdrive/MyDrive/Loras/playful_origins_v4/captions -> True
Output dir: /content/gdrive/MyDrive/Loras/playful_origins_v4/output -> True


In [12]:
!ls -R "/content/drive/MyDrive/Loras/playful_origins_v4"

/content/drive/MyDrive/Loras/playful_origins_v4:
captions  dataset  output

/content/drive/MyDrive/Loras/playful_origins_v4/captions:

/content/drive/MyDrive/Loras/playful_origins_v4/dataset:

/content/drive/MyDrive/Loras/playful_origins_v4/output:


In [15]:
import os

folder = "/content/drive/MyDrive/Loras/playful_origins_v4/dataset"
files = os.listdir(folder)
print(f"Found {len(files)} files in dataset:")
for f in files[:10]:
    print("  ", f)


Found 0 files in dataset:


In [30]:
!nvidia-smi


Thu Oct  9 00:46:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
# @title
%cd /content


/content


In [37]:
!wget -q "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors" -O /content/model.safetensors
!wget -q -O /content/config.yaml https://github.com/CompVis/stable-diffusion/raw/main/configs/stable-diffusion/v1-inference.yaml

# Convert the safetensors model to diffusers format
!mkdir -p /content/stable-diffusion-v1-5
!python /content/diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path /content/model.safetensors \
  --dump_path /content/stable-diffusion-v1-5 \
  --original_config_file /content/config.yaml \
  --from_safetensors


python3: can't open file '/content/diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py': [Errno 2] No such file or directory


In [38]:
!pip install diffusers==0.27.2 transformers accelerate safetensors --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 136.6 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.35.1
    Uninstalling diffusers-0.35.1:
      Successfully uninstalled diffusers-0.35.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [44]:
# Download the conversion script manually (TheLastBen version)
!wget -q -O /content/convert_original_stable_diffusion_to_diffusers.py \
  https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/convert_original_stable_diffusion_to_diffusers.py

# Make sure your output folder exists
!mkdir -p /content/stable-diffusion-v1-5

# Convert the model to diffusers format
!python /content/convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path /content/model.safetensors \
  --dump_path /content/stable-diffusion-v1-5 \
  --original_config_file /content/config.yaml \
  --from_safetensors



In [46]:
!ls -lh /content/



total 4.0G
-rw-r--r-- 1 root root 1.9K Oct  9 01:03 config.yaml
-rw-r--r-- 1 root root    0 Oct  9 01:06 convert_original_stable_diffusion_to_diffusers.py
drwxr-xr-x 3 root root 4.0K Oct  9 00:01 drive
drwx------ 5 root root 4.0K Oct  9 00:01 gdrive
-rw-r--r-- 1 root root 4.0G Oct  9 01:02 model.safetensors
drwxr-xr-x 2 root root 4.0K Oct  9 00:03 __pycache__
drwxr-xr-x 1 root root 4.0K Oct  7 13:38 sample_data
-rw-r--r-- 1 root root 7.7K Oct  9 00:03 smart_crop.py
drwxr-xr-x 3 root root 4.0K Oct  9 00:58 stable-diffusion-custom
drwxr-xr-x 2 root root 4.0K Oct  9 01:03 stable-diffusion-v1-5


In [1]:
# Download the working conversion script from Hugging Face diffusers
!wget -O /content/convert_original_stable_diffusion_to_diffusers.py \
  https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py

# Make sure it’s executable
!chmod +x /content/convert_original_stable_diffusion_to_diffusers.py


--2025-10-09 01:16:00--  https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7020 (6.9K) [text/plain]
Saving to: ‘/content/convert_original_stable_diffusion_to_diffusers.py’

/content/convert_or 100%[===================>]   6.86K  --.-KB/s    in 0s      

2025-10-09 01:16:00 (41.8 MB/s) - ‘/content/convert_original_stable_diffusion_to_diffusers.py’ saved [7020/7020]



In [2]:
!python /content/convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path /content/model.safetensors \
  --dump_path /content/stable-diffusion-v1-5 \
  --original_config_file /content/config.yaml \
  --from_safetensors


2025-10-09 01:16:47.093415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759972607.118615    1011 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759972607.125979    1011 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759972607.145291    1011 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759972607.145334    1011 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759972607.145339    1011 computation_placer.cc:177] computation placer alr

In [4]:
!ls -lh /content



total 12K
-rwxr-xr-x 1 root root 6.9K Oct  9 01:16 convert_original_stable_diffusion_to_diffusers.py
drwxr-xr-x 1 root root 4.0K Oct  7 13:38 sample_data


In [42]:
!mkdir -p /content/stable-diffusion-v1-5
!python /content/convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path /content/model.safetensors \
  --dump_path /content/stable-diffusion-v1-5 \
  --original_config_file /content/config.yaml \
  --from_safetensors


In [11]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=87f6f5b4687591bdf6b621b035bc9125ef3425fc00a42bd4bb648e4afc7f8027
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [25]:
!pip install diffusers==0.27.2 transformers==4.37.2 accelerate==0.29.3 safetensors==0.4.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.6.2
    Uninstalling safetensors-0.6.2:
      Successfully uninstalled safetensors-0.6.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.35.1
    Uninstalling diffusers-0.35.1:
      Successfully uninstalled diffusers-0.35.1
  Attempting uninsta

In [26]:
!wget -O /content/convert_original_stable_diffusion_to_diffusers.py \
  https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py


--2025-10-09 02:53:01--  https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7020 (6.9K) [text/plain]
Saving to: ‘/content/convert_original_stable_diffusion_to_diffusers.py’

/content/convert_or 100%[===================>]   6.86K  --.-KB/s    in 0s      

2025-10-09 02:53:01 (22.2 MB/s) - ‘/content/convert_original_stable_diffusion_to_diffusers.py’ saved [7020/7020]



In [27]:
!ls -lh /content/convert_original_stable_diffusion_to_diffusers.py


-rwxr-xr-x 1 root root 6.9K Oct  9 02:53 /content/convert_original_stable_diffusion_to_diffusers.py


In [30]:
!ls -lh /content



total 7.2G
-rwxr-xr-x 1 root root 6.9K Oct  9 02:53  convert_original_stable_diffusion_to_diffusers.py
-rw-r--r-- 1 root root 7.0K Oct  9 02:22 'det (1).py'
-rw-r--r-- 1 root root 7.0K Oct  9 01:34  det.py
-rw-r--r-- 1 root root 7.2G Oct  9 01:24  model.safetensors
drwxr-xr-x 1 root root 4.0K Oct  7 13:38  sample_data
drwxr-xr-x 2 root root 4.0K Oct  9 02:54  stable-diffusion-custom
drwxr-xr-x 9 root root 4.0K Oct  9 01:28  stable-diffusion-v1-5


In [36]:
import os, time
from IPython.display import clear_output

MODEL_PATH = "/content/model.safetensors"
sftnsr = "--from_safetensors"

print("Custom model version set manually to 1.5 — skipping detection.")
clear_output()

# Convert from .safetensors to Diffusers
!wget -q -O /content/config.yaml https://github.com/CompVis/stable-diffusion/raw/main/configs/stable-diffusion/v1-inference.yaml
!python /content/convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path "$MODEL_PATH" \
  --dump_path /content/stable-diffusion-custom \
  --original_config_file /content/config.yaml \
  $sftnsr
!rm /content/config.yaml

# Confirm conversion result
if os.path.exists('/content/stable-diffusion-custom/unet/diffusion_pytorch_model.bin'):
    clear_output()
    print("✅ DONE! Model converted successfully.")
else:
    print("❌ Conversion failed — check paths or file type.")



/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2025-10-09 03:06:13.760905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759979173.785594   27511 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759979173.793160   27511 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759979173.811316   27511 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:

In [37]:
!pip install accelerate==0.21.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3


In [34]:
!pip install huggingface-hub==0.25.2 diffusers==0.25.0 transformers==4.37.2 accelerate==0.29.3 safetensors==0.4.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.27.2
    Uninstalling diffusers-0.27.2:
      Successfully uninstalled diffusers-0.27.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.47.2 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.25.2 which is incompatible.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.


In [35]:
!pip show diffusers huggingface-hub | grep Version


Version: 0.25.0
Version: 0.25.2


In [32]:
!pip install huggingface-hub==0.19.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.35.3
    Uninstalling huggingface-hub-0.35.3:
      Successfully uninstalled huggingface-hub-0.35.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
diffusers 0.27.2 requires huggingface-hub>=0.20.2, but you have huggingface-hub 0.19.4 which is incompatible.
gradio 5.47.2 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.19.4 which is incompatible.
sentence-transformers 5.1.1 requires huggingface-hub>=0.20.0, but you have huggingface-hub 0.19.4 which is incompatible.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.
peft 0.17.1 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.19.4 which i

In [38]:
!ls -lh /content/stable-diffusion-custom


total 0


In [41]:
!python convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path /content/ComfyUI/models/checkpoints/Playful_OriginsV4.0.ckpt \
  --dump_path /content/stable-diffusion-custom \
  --device cpu


/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2025-10-09 04:08:40.553269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759982920.577492   42501 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759982920.584847   42501 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759982920.602937   42501 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:

# Model Download

In [48]:
#@title Model Download
from IPython.utils import capture
import os
import time
import urllib.request
from IPython.display import clear_output

# Set the model version you want to use
Model_Version = "1.5"  #@param ["1.5", "V2.1-512px", "V2.1-768px"]

# Optional: If you want to load a model from Hugging Face directly
Path_to_HuggingFace = "runwayml/stable-diffusion-v1-5"  #@param {type:"string"}

# These will be ignored unless you're using a custom model
MODEL_PATH = ""  #@param {type:"string"}
MODEL_LINK = ""  #@param {type:"string"}

# This sets the model destination path based on version
if Model_Version == "1.5":
    MODEL_NAME = "runwayml/stable-diffusion-v1-5"
    print("✅ Using Stable Diffusion v1.5 from Hugging Face:", MODEL_NAME)

elif Model_Version == "V2.1-512px":
    MODEL_NAME = "stabilityai/stable-diffusion-2-1-base"
    print("✅ Using Stable Diffusion v2.1-512px from Hugging Face:", MODEL_NAME)

elif Model_Version == "V2.1-768px":
    MODEL_NAME = "stabilityai/stable-diffusion-2-1"
    print("✅ Using Stable Diffusion v2.1-768px from Hugging Face:", MODEL_NAME)

# Override MODEL_NAME if a custom HF path is given
if Path_to_HuggingFace:
    MODEL_NAME = Path_to_HuggingFace
    print("📦 Custom Hugging Face model path set:", MODEL_NAME)

# At this point, MODEL_NAME is ready to be used by the training or inference cells.


✅ Using Stable Diffusion v1.5 from Hugging Face: runwayml/stable-diffusion-v1-5
📦 Custom Hugging Face model path set: runwayml/stable-diffusion-v1-5


In [23]:
INSTANCE_DIR = "/content/gdrive/MyDrive/Loras/playful_origins_v4/dataset"
CAPTIONS_DIR = "/content/gdrive/MyDrive/Loras/playful_origins_v4/captions"
OUTPUT_DIR   = "/content/gdrive/MyDrive/Loras/playful_origins_v4/output"

print("✅ Paths updated:")
print("INSTANCE_DIR:", INSTANCE_DIR)
print("CAPTIONS_DIR:", CAPTIONS_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)


✅ Paths updated:
INSTANCE_DIR: /content/gdrive/MyDrive/Loras/playful_origins_v4/dataset
CAPTIONS_DIR: /content/gdrive/MyDrive/Loras/playful_origins_v4/captions
OUTPUT_DIR: /content/gdrive/MyDrive/Loras/playful_origins_v4/output


In [24]:
!ls "$INSTANCE_DIR" | head -n 10


IMG_6276.jpg
IMG_6276.txt
IMG_7935.PNG
IMG_7935.txt
IMG_7936.PNG
IMG_7936.txt
IMG_7937.JPG
IMG_7937.txt
IMG_7940.png
IMG_7940.txt


# Dreambooth

In [ ]:
import os
from IPython.display import clear_output
from IPython.utils import capture
from os import listdir
from os.path import isfile
from subprocess import check_output
import wget
import time

#@markdown #Create/Load a Session

try:
  MODEL_NAME
  pass
except:
  MODEL_NAME=""

PT=""

Session_Name = "" #@param{type: 'string'}
while Session_Name=="":
  print('[1;31mInput the Session Name:')
  Session_Name=input('')
Session_Name=Session_Name.replace(" ","_")

#@markdown - Enter the session name, it if it exists, it will load it, otherwise it'll create an new session.

Session_Link_optional = "" #@param{type: 'string'}

#@markdown - Import a session from another gdrive, the shared gdrive link must point to the specific session's folder that contains the trained CKPT, remove any intermediary CKPT if any.

WORKSPACE='/content/gdrive/MyDrive/Fast-Dreambooth'

if Session_Link_optional !="":
  print('[1;32mDownloading session...')
  with capture.capture_output() as cap:
    %cd /content
    if not os.path.exists(str(WORKSPACE+'/Sessions')):
      %mkdir -p $WORKSPACE'/Sessions'
      time.sleep(1)
    %cd $WORKSPACE'/Sessions'
    !gdown --folder --remaining-ok -O $Session_Name  $Session_Link_optional
    %cd $Session_Name
    !rm -r instance_images
    !unzip instance_images.zip
    !rm -r captions
    !unzip captions.zip
    %cd /content


INSTANCE_NAME=Session_Name
OUTPUT_DIR="/content/models/"+Session_Name
SESSION_DIR=WORKSPACE+'/Sessions/'+Session_Name
INSTANCE_DIR=SESSION_DIR+'/instance_images'
CAPTIONS_DIR=SESSION_DIR+'/captions'
MDLPTH=str(SESSION_DIR+"/"+Session_Name+'.ckpt')

if os.path.exists(str(SESSION_DIR)):
  mdls=[ckpt for ckpt in listdir(SESSION_DIR) if ckpt.split(".")[-1]=="ckpt"]
  if not os.path.exists(MDLPTH) and '.ckpt' in str(mdls):

    def f(n):
      k=0
      for i in mdls:
        if k==n:
          !mv "$SESSION_DIR/$i" $MDLPTH
        k=k+1

    k=0
    print('[1;33mNo final checkpoint model found, select which intermediary checkpoint to use, enter only the number, (000 to skip):\n[1;34m')

    for i in mdls:
      print(str(k)+'- '+i)
      k=k+1
    n=input()
    while int(n)>k-1:
      n=input()
    if n!="000":
      f(int(n))
      print('[1;32mUsing the model '+ mdls[int(n)]+" ...")
      time.sleep(2)
    else:
      print('[1;32mSkipping the intermediary checkpoints.')
    del n

with capture.capture_output() as cap:
  %cd /content
  resume=False

if os.path.exists(str(SESSION_DIR)) and not os.path.exists(MDLPTH):
  print('[1;32mLoading session with no previous model, using the original model or the custom downloaded model')
  if MODEL_NAME=="":
    print('[1;31mNo model found, use the "Model Download" cell to download a model.')
  else:
    print('[1;32mSession Loaded, proceed to uploading instance images')

elif os.path.exists(MDLPTH):
  print('[1;32mSession found, loading the trained model ...')
  wget.download('https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/det.py')
  print('[1;33mDetecting model version...')
  Model_Version=check_output('python det.py --MODEL_PATH '+MDLPTH, shell=True).decode('utf-8').replace('\n', '')
  clear_output()
  print('[1;32m'+Model_Version+' Detected')
  !rm det.py
  if Model_Version=='1.5':
    !wget -q -O config.yaml https://github.com/CompVis/stable-diffusion/raw/main/configs/stable-diffusion/v1-inference.yaml
    print('[1;32mSession found, loading the trained model ...')
    !python /content/diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path $MDLPTH --dump_path "$OUTPUT_DIR" --original_config_file config.yaml
    !rm /content/config.yaml

  elif Model_Version=='V2.1-512px':
    !wget -q -O convertodiff.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/convertodiffv2.py
    print('[1;32mSession found, loading the trained model ...')
    !python /content/convertodiff.py "$MDLPTH" "$OUTPUT_DIR" --v2 --reference_model stabilityai/stable-diffusion-2-1-base
    !rm /content/convertodiff.py

  elif Model_Version=='V2.1-768px':
    !wget -q -O convertodiff.py https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dreambooth/convertodiffv2-768.py
    print('[1;32mSession found, loading the trained model ...')
    !python /content/convertodiff.py "$MDLPTH" "$OUTPUT_DIR" --v2 --reference_model stabilityai/stable-diffusion-2-1
    !rm /content/convertodiff.py


  if os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
    resume=True
    clear_output()
    print('[1;32mSession loaded.')
  else:
    if not os.path.exists(OUTPUT_DIR+'/unet/diffusion_pytorch_model.bin'):
      print('[1;31mConversion error, if the error persists, remove the CKPT file from the current session folder')

elif not os.path.exists(str(SESSION_DIR)):
    %mkdir -p "$INSTANCE_DIR"
    print('[1;32mCreating session...')
    if MODEL_NAME=="":
      print('[1;31mNo model found, use the "Model Download" cell to download a model.')
    else:
      print('[1;32mSession created, proceed to uploading instance images')

    #@markdown

    #@markdown # The most important step is to rename the instance pictures of each subject to a unique unknown identifier, example :
    #@markdown - If you have 10 pictures of yourself, simply select them all and rename only one to the chosen identifier for example : phtmejhn, the files would be : phtmejhn (1).jpg, phtmejhn (2).png ....etc then upload them, do the same for other people or objects with a different identifier, and that's it.
    #@markdown - Checkout this example : https://i.imgur.com/d2lD3rz.jpeg

In [6]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=5f704027ca7a49f9140c056cac5a97119db91379095bc3e9612913ec767dd199
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [7]:
import os
import wget
from IPython.utils import capture

In [49]:
import shutil
from google.colab import files
import time
from PIL import Image
from tqdm import tqdm
import ipywidgets as widgets
from io import BytesIO
import wget

with capture.capture_output() as cap:
  %cd /content
  if not os.path.exists("/content/smart_crop.py"):
    wget.download('https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dreambooth/smart_crop.py')
  from smart_crop import *

#@markdown #Instance Images
#@markdown ----

#@markdown
#@markdown - Run the cell to upload the instance pictures.
#@markdown - You can add `external captions` in txt files by simply giving each txt file the same name as the instance image, for example dikgur (1).jpg and dikgur (1).txt, and upload them here, to use the external captions, check the box "external_captions" in the training cell. `All the images must have one same extension` jpg or png or....etc

Remove_existing_instance_images= True #@param{type: 'boolean'}
#@markdown - Uncheck the box to keep the existing instance images.

if Remove_existing_instance_images:
  if os.path.exists(str(INSTANCE_DIR)):
    !rm -r "$INSTANCE_DIR"
  if os.path.exists(str(CAPTIONS_DIR)):
    !rm -r "$CAPTIONS_DIR"

if not os.path.exists(str(INSTANCE_DIR)):
  %mkdir -p "$INSTANCE_DIR"
if not os.path.exists(str(CAPTIONS_DIR)):
  %mkdir -p "$CAPTIONS_DIR"

if os.path.exists(INSTANCE_DIR+"/.ipynb_checkpoints"):
  %rm -r $INSTANCE_DIR"/.ipynb_checkpoints"


IMAGES_FOLDER_OPTIONAL="/content/drive/MyDrive/Loras/playful_origins_v4/dataset" #@param{type: 'string'}

#@markdown - If you prefer to specify directly the folder of the pictures instead of uploading, this will add the pictures to the existing (if any) instance images. Leave EMPTY to upload.

Smart_Crop_images= True #@param{type: 'boolean'}
Crop_size = 512 #@param ["512", "576", "640", "704", "768", "832", "896", "960", "1024"] {type:"raw"}

#@markdown - Smart crop the images without manual intervention.

while IMAGES_FOLDER_OPTIONAL !="" and not os.path.exists(str(IMAGES_FOLDER_OPTIONAL)):
  print('[1;31mThe image folder specified does not exist, use the colab file explorer to copy the path :')
  IMAGES_FOLDER_OPTIONAL=input('')

if IMAGES_FOLDER_OPTIONAL!="":
  if os.path.exists(IMAGES_FOLDER_OPTIONAL+"/.ipynb_checkpoints"):
    %rm -r "$IMAGES_FOLDER_OPTIONAL""/.ipynb_checkpoints"

  with capture.capture_output() as cap:
    !mv $IMAGES_FOLDER_OPTIONAL/*.txt $CAPTIONS_DIR
  if Smart_Crop_images:
    for filename in tqdm(os.listdir(IMAGES_FOLDER_OPTIONAL), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
      extension = filename.split(".")[-1]
      identifier=filename.split(".")[0]
      new_path_with_file = os.path.join(INSTANCE_DIR, filename)
      file = Image.open(IMAGES_FOLDER_OPTIONAL+"/"+filename)
      width, height = file.size
      if file.size !=(Crop_size, Crop_size):
        image=crop_image(file, Crop_size)
        if extension.upper()=="JPG" or extension.upper()=="jpg":
            image[0] = image[0].convert("RGB")
            image[0].save(new_path_with_file, format="JPEG", quality = 100)
        else:
            image[0].save(new_path_with_file, format=extension.upper())
      else:
        !cp "$IMAGES_FOLDER_OPTIONAL/$filename" "$INSTANCE_DIR"

  else:
    for filename in tqdm(os.listdir(IMAGES_FOLDER_OPTIONAL), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
      %cp -r "$IMAGES_FOLDER_OPTIONAL/$filename" "$INSTANCE_DIR"

  print('\n[1;32mDone, proceed to the next cell')


elif IMAGES_FOLDER_OPTIONAL =="":
  up=""
  uploaded = files.upload()
  for filename in uploaded.keys():
    if filename.split(".")[-1]=="txt":
      shutil.move(filename, CAPTIONS_DIR)
    up=[filename for filename in uploaded.keys() if filename.split(".")[-1]!="txt"]
  if Smart_Crop_images:
    for filename in tqdm(up, bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
      shutil.move(filename, INSTANCE_DIR)
      extension = filename.split(".")[-1]
      identifier=filename.split(".")[0]
      new_path_with_file = os.path.join(INSTANCE_DIR, filename)
      file = Image.open(new_path_with_file)
      width, height = file.size
      if file.size !=(Crop_size, Crop_size):
        image=crop_image(file, Crop_size)
        if extension.upper()=="JPG" or extension.upper()=="jpg":
            image[0] = image[0].convert("RGB")
            image[0].save(new_path_with_file, format="JPEG", quality = 100)
        else:
            image[0].save(new_path_with_file, format=extension.upper())
      clear_output()
  else:
    for filename in tqdm(uploaded.keys(), bar_format='  |{bar:15}| {n_fmt}/{total_fmt} Uploaded'):
      shutil.move(filename, INSTANCE_DIR)
      clear_output()
  print('\n[1;32mDone, proceed to the next cell')

with capture.capture_output() as cap:
  %cd "$INSTANCE_DIR"
  !find . -name "* *" -type f | rename 's/ /-/g'
  %cd "$CAPTIONS_DIR"
  !find . -name "* *" -type f | rename 's/ /-/g'

  %cd $SESSION_DIR
  !rm instance_images.zip captions.zip
  !zip -r instance_images instance_images
  !zip -r captions captions
  %cd /content

  |               | 0/0 Uploaded



Done, proceed to the next cell


In [17]:
print("INSTANCE_DIR:", INSTANCE_DIR)
!ls "$INSTANCE_DIR" | head -n 10


INSTANCE_DIR: /content/drive/MyDrive/Loras/playful_origins_v4/dataset


In [22]:
!ls "/content/gdrive/MyDrive/Loras/playful_origins_v4/dataset" | head -n 10



IMG_6276.jpg
IMG_6276.txt
IMG_7935.PNG
IMG_7935.txt
IMG_7936.PNG
IMG_7936.txt
IMG_7937.JPG
IMG_7937.txt
IMG_7940.png
IMG_7940.txt


In [10]:
import os

folder = "/content/drive/MyDrive/Loras/playful_origins_v4/dataset"
files = os.listdir(folder)
print(f"Found {len(files)} files in dataset:")
for f in files[:10]:
    print("  ", f)


Found 0 files in dataset:


In [ ]:
import ipywidgets as widgets
from io import BytesIO
#@markdown #Captions (optional)

#@markdown - Open a tool to manually `create` captions or edit existing captions of the instance images, do not use captions when training on a face.

paths=""
out=""
widgets_l=""
clear_output()
def Caption(path):
    if path!="Select an instance image to caption":

      name = os.path.splitext(os.path.basename(path))[0]
      ext=os.path.splitext(os.path.basename(path))[-1][1:]
      if ext=="jpg" or "JPG":
        ext="JPEG"

      if os.path.exists(CAPTIONS_DIR+"/"+name + '.txt'):
        with open(CAPTIONS_DIR+"/"+name + '.txt', 'r') as f:
            text = f.read()
      else:
        with open(CAPTIONS_DIR+"/"+name + '.txt', 'w') as f:
            f.write("")
            with open(CAPTIONS_DIR+"/"+name + '.txt', 'r') as f:
                text = f.read()

      img=Image.open(os.path.join(INSTANCE_DIR,path))
      img=img.convert("RGB")
      img=img.resize((420, 420))
      image_bytes = BytesIO()
      img.save(image_bytes, format=ext, qualiy=10)
      image_bytes.seek(0)
      image_data = image_bytes.read()
      img= image_data
      image = widgets.Image(
          value=img,
          width=420,
          height=420
      )
      text_area = widgets.Textarea(value=text, description='', disabled=False, layout={'width': '300px', 'height': '120px'})


      def update_text(text):
          with open(CAPTIONS_DIR+"/"+name + '.txt', 'w') as f:
              f.write(text)

      button = widgets.Button(description='Save', button_style='success')
      button.on_click(lambda b: update_text(text_area.value))

      return widgets.VBox([widgets.HBox([image, text_area, button])])


paths = os.listdir(INSTANCE_DIR)
widgets_l = widgets.Select(options=["Select an instance image to caption"]+paths, rows=25)


out = widgets.Output()

def click(change):
    with out:
        out.clear_output()
        display(Caption(change.new))

widgets_l.observe(click, names='value')
display(widgets.HBox([widgets_l, out]))

In [26]:
# ✅ Set your model output folder manually
OUTPUT_DIR = "/content/drive/MyDrive/Loras/playful_origins_v4/output"

# make sure it exists
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("✅ Output directory set to:", OUTPUT_DIR)
!ls -a "$OUTPUT_DIR"


✅ Output directory set to: /content/drive/MyDrive/Loras/playful_origins_v4/output
.  ..


In [33]:
!ls /content


drive  gdrive  __pycache__  sample_data  smart_crop.py


In [6]:
!find /content -type f \( -name "*.safetensors" -o -name "config.yaml" \)

In [7]:
# Download the Stable Diffusion v1.5 model weights
!wget -q -O /content/model.safetensors https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.safetensors

# Download the corresponding config file
!wget -q -O /content/config.yaml https://huggingface.co/runwayml/stable-diffusion-v1-5/raw/main/v1-inference.yaml


In [8]:
!mkdir -p /content/stable-diffusion-v1-5

!python /content/convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path "/content/model.safetensors" \
  --dump_path "/content/stable-diffusion-v1-5" \
  --original_config_file "/content/config.yaml" \
  --from_safetensors


2025-10-09 01:25:47.313547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759973147.343635    3212 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759973147.352839    3212 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759973147.374918    3212 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759973147.374969    3212 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759973147.374974    3212 computation_placer.cc:177] computation placer alr

In [9]:
!ls /content/stable-diffusion-v1-5

feature_extractor  safety_checker  text_encoder  unet
model_index.json   scheduler	   tokenizer	 vae


# Training

In [32]:
#@markdown ---
#@markdown # Start DreamBooth
#@markdown ---
import os
import time
import random
from IPython.display import clear_output
from google.colab import runtime
from subprocess import getoutput

# Cleanup .ipynb_checkpoints
for path in [INSTANCE_DIR, CAPTIONS_DIR]:
    chkpt_path = os.path.join(path, ".ipynb_checkpoints")
    if os.path.exists(chkpt_path):
        %rm -r "$chkpt_path"

# Initialize Resume
Resume_Training = False  #@param {type:"boolean"}
resume = False

if resume and not Resume_Training:
    print('\033[1;31mOverwrite previous model? "yes" to start new, "no" to resume training:')
    while True:
        ansres = input("").strip().lower()
        if ansres == 'no':
            Resume_Training = True
            break
        elif ansres == 'yes':
            Resume_Training = False
            resume = False
            break

while not Resume_Training and MODEL_NAME == "":
    print('\033[1;31mNo model found. Use the "Model Download" cell first.')
    time.sleep(5)

# Training config
UNet_Training_Steps = 1111  #@param {type:'number'}
UNet_Learning_Rate = 2e-6  #@param ["2e-5", "1e-5", "9e-6", "8e-6", "7e-6", "6e-6", "5e-6", "4e-6", "3e-6", "2e-6"] {type:"raw"}
Text_Encoder_Training_Steps = 350  #@param {type:'number'}
Text_Encoder_Learning_Rate = 1e-6  #@param ["2e-6", "1e-6", "8e-7", "6e-7", "5e-7", "4e-7"] {type:"raw"}
Offset_Noise = False  #@param {type:"boolean"}
External_Captions = True  #@param {type:"boolean"}
Resolution = "512"  #@param ["512", "576", "640", "704", "768", "832", "896", "960", "1024"]
Save_Checkpoint_Every_n_Steps = False  #@param {type:"boolean"}
Save_Checkpoint_Every = 500  #@param {type:'number'}
Start_saving_from_the_step = 500  #@param {type:'number'}
Disconnect_after_training = False  #@param {type:"boolean"}

# Derived flags
trnonltxt = "--train_only_text_encoder" if UNet_Training_Steps == 0 else ""
extrnlcptn = "--external_captions" if External_Captions else ""
ofstnse = "--offset_noise" if Offset_Noise else ""
Res = int(Resolution)
Seed = random.randint(1, 999999)
precision = "fp16"

# Handle checkpointing
if Save_Checkpoint_Every is None:
    Save_Checkpoint_Every = 500
if Start_saving_from_the_step is None or Start_saving_from_the_step < 200:
    Start_saving_from_the_step = Save_Checkpoint_Every
stpsv = Start_saving_from_the_step
stp = Save_Checkpoint_Every if Save_Checkpoint_Every_n_Steps else 0

# Resume handling
if Resume_Training and os.path.exists(f"{OUTPUT_DIR}/unet/diffusion_pytorch_model.bin"):
    MODEL_NAME = OUTPUT_DIR
    print("\033[1;32mResuming Training...\033[0m")
    resuming = "Yes"
else:
    resuming = ""

# Determine model version
V2 = False
if os.path.getsize(f"{MODEL_NAME}/text_encoder/pytorch_model.bin") > 670_901_463:
    V2 = True

# Handle memory
s = getoutput("nvidia-smi")
GCUNET = "--gradient_checkpointing" if (Res > 768 or V2 or 'A100' not in s) else ""

# Define session dir fallback
if "SESSION_DIR" not in globals():
    SESSION_DIR = "/content/sessions"
if "Session_Name" not in globals():
    Session_Name = INSTANCE_NAME

# Train text encoder
def dump_only_textenc():
    !accelerate launch /content/diffusers/examples/dreambooth/train_dreambooth.py \
    $trnonltxt \
    $extrnlcptn \
    $ofstnse \
    --image_captions_filename \
    --train_text_encoder \
    --dump_only_text_encoder \
    --pretrained_model_name_or_path="$MODEL_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --captions_dir="$CAPTIONS_DIR" \
    --instance_prompt="$PT" \
    --seed=$Seed \
    --resolution=$Res \
    --mixed_precision=$precision \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 $GCUNET \
    --use_8bit_adam \
    --learning_rate=$Text_Encoder_Learning_Rate \
    --lr_scheduler="linear" \
    --lr_warmup_steps=0 \
    --max_train_steps=$Text_Encoder_Training_Steps

# Train UNet
def train_only_unet():
    clear_output()
    if resuming == "Yes":
        print("\033[1;32mResuming Training...\033[0m")
    print("\033[1;33mTraining the UNet...\033[0m")
    !accelerate launch /content/diffusers/examples/dreambooth/train_dreambooth.py \
    $extrnlcptn \
    $ofstnse \
    --image_captions_filename \
    --train_only_unet \
    --save_starting_step=$stpsv \
    --save_n_steps=$stp \
    --Session_dir=$SESSION_DIR \
    --pretrained_model_name_or_path="$MODEL_NAME" \
    --instance_data_dir="$INSTANCE_DIR" \
    --output_dir="$OUTPUT_DIR" \
    --captions_dir="$CAPTIONS_DIR" \
    --instance_prompt="$PT" \
    --seed=$Seed \
    --resolution=$Res \
    --mixed_precision=$precision \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 $GCUNET \
    --use_8bit_adam \
    --learning_rate=$UNet_Learning_Rate \
    --lr_scheduler="linear" \
    --lr_warmup_steps=0 \
    --max_train_steps=$UNet_Training_Steps

# Run training
if Text_Encoder_Training_Steps > 0:
    print("\033[1;33mTraining the text encoder...\033[0m")
    if os.path.exists(f"{OUTPUT_DIR}/text_encoder_trained"):
        %rm -r "$OUTPUT_DIR/text_encoder_trained"
    dump_only_textenc()

if UNet_Training_Steps > 0:
    train_only_unet()

# Final conversion
if UNet_Training_Steps == 0 and Text_Encoder_Training_Steps == 0:
    print("\033[1;32mNothing to do")
else:
    final_ckpt = f"/content/models/{INSTANCE_NAME}/unet/diffusion_pytorch_model.bin"
    if os.path.exists(final_ckpt):
        prc = "--fp16" if precision == "fp16" else ""
        !python /content/diffusers/scripts/convertosdv2.py $prc "$OUTPUT_DIR" "$SESSION_DIR/$Session_Name.ckpt"
        clear_output()
        if os.path.exists(f"{SESSION_DIR}/{INSTANCE_NAME}.ckpt"):
            print("\033[1;32m✅ DONE — Your CKPT is saved to Google Drive in the sessions folder.")
            if Disconnect_after_training:
                time.sleep(20)
                runtime.unassign()
        else:
            print("\033[1;31m❌ Something went wrong during conversion.")
    else:
        print("\033[1;31m❌ No final model file found to convert.")


FileNotFoundError: [Errno 2] No such file or directory: 'runwayml/stable-diffusion-v1-5/text_encoder/pytorch_model.bin'

In [ ]:
#@markdown #Free Gdrive Space

#@markdown Display the list of sessions from your gdrive and choose which ones to remove.

import ipywidgets as widgets

Sessions=os.listdir("/content/gdrive/MyDrive/Fast-Dreambooth/Sessions")

s = widgets.Select(
    options=Sessions,
    rows=5,
    description='',
    disabled=False
)

out=widgets.Output()

d = widgets.Button(
    description='Remove',
    disabled=False,
    button_style='warning',
    tooltip='Removet the selected session',
    icon='warning'
)

def rem(d):
    with out:
        if s.value is not None:
            clear_output()
            print("[1;33mTHE SESSION [1;31m"+s.value+" [1;33mHAS BEEN REMOVED FROM YOUR GDRIVE")
            !rm -r '/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/{s.value}'
            s.options=os.listdir("/content/gdrive/MyDrive/Fast-Dreambooth/Sessions")
        else:
            d.close()
            s.close()
            clear_output()
            print("[1;32mNOTHING TO REMOVE")

d.on_click(rem)
if s.value is not None:
    display(s,d,out)
else:
    print("[1;32mNOTHING TO REMOVE")